In [ ]:
import torch
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import shap
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# ResNet50 모델 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # GPU 사용시를 상정하여 작성
model = models.resnet50(weights=None) # 가중치 없는 모델
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 3) # 클래스 3개
model.load_state_dict(torch.load("best_model.pth", map_location=device)) # 저장된 모델 불러오기기 
model.to(device)
model.eval() # 모델을 평가 모드로 전환